In [ ]:
import google.generativeai as genai

import json
import ast

genai.configure(api_key='')
model = genai.GenerativeModel('gemini-pro')

response = model.generate_content("Write a story about a magic backpack.")
print(response.text)

In [ ]:
# read training files
prompts = []
with open('training_data/train_1m', 'r', encoding='utf-8') as f:
    content = f.readlines()
    prompt = ""
    for line in content:
        if line == '<end>\n':
            prompts.append(prompt)
            prompt = ""
            continue
        prompt += line

SUF = ['', '_CoT', '_ToT']
_suf = SUF[2]

tot_prompt = "Imagine five different experts answering this question. All experts will write down 1 step of their thinking and then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they're wrong at any point, then they leave. And show me the result in the end. \nThe question is:\n\nYou are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least five movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. The movies should be before 2001.\n"

cot_prompt = "You are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least five movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. Think step by step and make sure. The movies should be before 2001.\n"

plain_prompt = "You are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least five movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. The movies should be before 2001.\n"

j = 0
with open(f"training_data/output_gpt_1m{_suf}_pro", "w", encoding='utf-8') as f:
    with open(f"training_data/output_gpt_1m{_suf}_pro.json", "w", encoding='utf-8') as fj:
        result = []

        for prompt in prompts:
            j += 1
            print(j)
            for i in range(2):
                message = [
                    {"role": "system", "content": cot_prompt},
                    {"role": "user", "content": prompt}
                ]
                completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=message
                )

                message.append({"role": "assistant", "content": completion.choices[0].message.content})
                message.append({"role": "user",
                                "content": "Output the influence path in the format of python list object. Example output:\n[movie name1,\nmovie name2,\nmovie name3,\nmovie name4,\n...\ntarget movie name]\n"})
                completion = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=message
                )

                f.write(prompt)
                f.write("\nInfluence path:\n")
                influence_path = completion.choices[0].message.content

                try:
                    influence_path_get = influence_path[influence_path.index("["):influence_path.index("]") + 1]
                    influence_path_lst = ast.literal_eval(influence_path_get)
                except:
                    print(f"Error Output:{influence_path}")
                    influence_path_lst = []

                for influence in influence_path_lst:
                    f.write(f"{influence}\n")
                f.write("<end>\n")

                if i == 0:
                    result.append([influence_path_lst])
                elif i == 1:
                    result[-1].append(influence_path_lst)
        json.dump(result, fj)